<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_ringworld_structural_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_ringworld_structural.py

Full training pipeline for RingworldAI with dtype fix in uncertainty heatmap:

1. Synthetic dataset of 5 ringworld parameters → 3 stability & efficiency metrics
2. Overflow‐safe normalization
3. Physics‐informed residuals enforcing structural, resource, and dynamic constraints
4. MLP with LayerNorm & Dropout
5. MC‐Dropout inference for uncertainty quantification
6. Training loop with AdamW, ReduceLROnPlateau, gradient clipping, early stopping
7. Visualizations: training curves, scatter plots, uncertainty heatmap
"""

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. Synthetic Ringworld Dataset
# ------------------------------------------------------------------------------
class RingworldDataset(Dataset):
    def __init__(self, n_samples=6000, seed=0):
        np.random.seed(seed)
        # Inputs: R (m), thickness t (m), yield_strength σy (Pa),
        # density ρ (kg/m³), angular_speed ω (rad/s)
        R  = np.random.uniform(1e7, 1e8,   (n_samples,1)).astype(np.float64)
        t  = np.random.uniform(1e2, 1e3,    (n_samples,1)).astype(np.float64)
        σy = np.random.uniform(1e8, 1e10,  (n_samples,1)).astype(np.float64)
        ρ  = np.random.uniform(1e3, 8e3,    (n_samples,1)).astype(np.float64)
        ω  = np.random.uniform(1e-4, 1e-3,  (n_samples,1)).astype(np.float64)

        X_raw = np.hstack([R, t, σy, ρ, ω])

        # Targets (toy physics):
        sm = σy / (ρ * ω**2 * R**2)                   # structural_margin
        re = R / (ρ * t)                             # resource_efficiency
        ds = 1.0 / (ω * np.sqrt(R))                  # dynamic_stability
        Y_raw = np.hstack([sm, re, ds]).astype(np.float64)
        Y_raw += 0.05 * np.std(Y_raw, axis=0) * np.random.randn(*Y_raw.shape)

        # Overflow‐safe normalization
        self.X_mean = X_raw.mean(0)
        with np.errstate(over='ignore'):
            self.X_std = X_raw.std(0) + 1e-6
        self.Y_mean = Y_raw.mean(0)
        with np.errstate(over='ignore'):
            self.Y_std = Y_raw.std(0) + 1e-6

        # Standardize and cast back to float32
        self.X = ((X_raw - self.X_mean) / self.X_std).astype(np.float32)
        self.Y = ((Y_raw - self.Y_mean) / self.Y_std).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.from_numpy(self.Y[idx])

# ------------------------------------------------------------------------------
# 2. Model Definition
# ------------------------------------------------------------------------------
class RingworldAI(nn.Module):
    def __init__(self, input_dim=5, hidden_dims=(64,64), output_dim=3, p_drop=0.1):
        super().__init__()
        layers, dim = [], input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(dim, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics-Informed Residual
# ------------------------------------------------------------------------------
def physics_residual(pred, inp, stats):
    X_den = inp * stats['X_std'] + stats['X_mean']
    Y_den = pred * stats['Y_std'] + stats['Y_mean']
    sm_pred, re_pred, ds_pred = Y_den.t()
    R, t, σy, ρ, ω = X_den.t()

    sm_true = σy / (ρ * ω**2 * R**2)
    re_true = R / (ρ * t)
    ds_true = 1.0 / (ω * torch.sqrt(R))

    return (
        nn.MSELoss()(sm_pred, sm_true) +
        nn.MSELoss()(re_pred, re_true) +
        nn.MSELoss()(ds_pred, ds_true)
    )

def total_loss(pred, true, inp, stats, λ=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, inp, stats)
    return mse + λ * phys, mse, phys

# ------------------------------------------------------------------------------
# 4. MC-Dropout Inference
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, x, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(x))
    arr  = torch.stack(preds, dim=0)
    return arr.mean(0), arr.std(0)

# ------------------------------------------------------------------------------
# 5. Training Loop
# ------------------------------------------------------------------------------
def train(model, train_ld, val_ld, stats, device,
          lr=1e-3, wd=1e-5, λ=1.0, max_epochs=150, patience=15):
    model.to(device)
    opt   = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', factor=0.5, patience=5)
    best_val, wait = float('inf'), 0
    history = {'train': [], 'val': []}

    for epoch in range(1, max_epochs+1):
        # training
        model.train()
        t_loss = 0
        for xb, yb in train_ld:
            xb, yb = xb.to(device), yb.to(device)
            pred, _, _ = total_loss(model(xb), yb, xb, stats, λ)
            opt.zero_grad(); pred.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            t_loss += pred.item() * xb.size(0)
        t_loss /= len(train_ld.dataset)

        # validation
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for xb, yb in val_ld:
                xb, yb = xb.to(device), yb.to(device)
                pred, _, _ = total_loss(model(xb), yb, xb, stats, λ)
                v_loss += pred.item() * xb.size(0)
        v_loss /= len(val_ld.dataset)

        sched.step(v_loss)
        history['train'].append(t_loss)
        history['val'].append(v_loss)
        print(f"Epoch {epoch:03d} | Train {t_loss:.4e} | Val {v_loss:.4e}")

        if v_loss < best_val - 1e-6:
            best_val, wait = v_loss, 0
            torch.save(model.state_dict(), "best_ringworld_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

    model.load_state_dict(torch.load("best_ringworld_ai.pth"))
    return history

# ------------------------------------------------------------------------------
# 6. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_history(h):
    plt.figure()
    plt.plot(h['train'], label='Train')
    plt.plot(h['val'],   label='Val')
    plt.xlabel("Epoch"); plt.ylabel("Loss")
    plt.legend(); plt.title("Training Curve"); plt.show()

def plot_scatter(y_true, y_pred, name):
    plt.figure()
    plt.scatter(y_true, y_pred, s=5, alpha=0.6)
    m, M = y_true.min(), y_true.max()
    plt.plot([m,M],[m,M],'r--')
    plt.xlabel(f"True {name}"); plt.ylabel(f"Pred {name}")
    plt.title(name); plt.show()

def plot_uncertainty_heatmap(model, stats_np, device):
    vals_R = np.linspace(1e7, 1e8, 100)
    vals_ω = np.linspace(1e-4, 1e-3, 100)
    RR, WW = np.meshgrid(vals_R, vals_ω)
    grid = np.zeros((RR.size,5), dtype=np.float32)
    grid[:,0], grid[:,4] = RR.ravel().astype(np.float32), WW.ravel().astype(np.float32)
    for i in (1,2,3):
        grid[:,i] = stats_np['X_mean'][i].astype(np.float32)

    # Cast to float32 here
    Xn = torch.from_numpy(grid).float().to(device)
    mean, std = mc_dropout_predict(model, Xn, T=100)
    std_sm = std[:,0].cpu().numpy().reshape(RR.shape)

    plt.figure(figsize=(6,5))
    plt.pcolormesh(RR, WW, std_sm, cmap='viridis', shading='auto')
    plt.colorbar(label='Std(structural_margin)')
    plt.xlabel('R (m)'); plt.ylabel('ω (rad/s)')
    plt.title('Uncertainty Heatmap: Structural Margin'); plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ds = RingworldDataset(6000)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, dtype=torch.float32, device=device),
        'X_std' : torch.tensor(ds.X_std,  dtype=torch.float32, device=device),
        'Y_mean': torch.tensor(ds.Y_mean, dtype=torch.float32, device=device),
        'Y_std' : torch.tensor(ds.Y_std,  dtype=torch.float32, device=device),
    }
    stats_np = {
        'X_mean': ds.X_mean, 'X_std': ds.X_std,
        'Y_mean': ds.Y_mean, 'Y_std': ds.Y_std,
    }

    n_val = int(0.2 * len(ds))
    tr, va = random_split(ds, [len(ds)-n_val, n_val])
    tr_ld = DataLoader(tr, batch_size=128, shuffle=True)
    va_ld = DataLoader(va, batch_size=256)

    model   = RingworldAI().to(device)
    history = train(model, tr_ld, va_ld, stats, device)

    plot_history(history)

    X_all = torch.from_numpy(ds.X).to(device)
    with torch.no_grad():
        Yp = model(X_all).cpu().numpy()
    Yt = ds.Y
    for i, name in enumerate(['structural_margin','resource_efficiency','dynamic_stability']):
        plot_scatter(Yt[:,i], Yp[:,i], name)

    plot_uncertainty_heatmap(model, stats_np, device)